# Введение в MapReduce модель на Python


In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [ ]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [ ]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [ ]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [ ]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [ ]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [ ]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [ ]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [ ]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [ ]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [ ]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.5987522744933476),
 (1, 1.5987522744933476),
 (2, 1.5987522744933476),
 (3, 1.5987522744933476),
 (4, 1.5987522744933476)]

## Inverted index

In [ ]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('is', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('it', ['0', '1', '2']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [ ]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [ ]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('it', 18)]),
 (1, [('banana', 2), ('is', 18), ('what', 10)])]

## TeraSort

In [ ]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.003581110070612459),
   (None, 0.059226068424185985),
   (None, 0.09637847188836224),
   (None, 0.15722779700252032),
   (None, 0.21437422874777923),
   (None, 0.2393372231746721),
   (None, 0.2831335025224506),
   (None, 0.2926907282827602),
   (None, 0.3322742834524207),
   (None, 0.34474769991995413),
   (None, 0.36351367386224653),
   (None, 0.39836579738488886),
   (None, 0.4122338533497053),
   (None, 0.4298450810309804),
   (None, 0.4753146139166151)]),
 (1,
  [(None, 0.5603922229968513),
   (None, 0.5664923070927802),
   (None, 0.6001731494029006),
   (None, 0.6299992041240836),
   (None, 0.6731321104363888),
   (None, 0.7036490125393071),
   (None, 0.7036801029105392),
   (None, 0.8261403284806825),
   (None, 0.8341704743669328),
   (None, 0.8498491314147857),
   (None, 0.8989169174971396),
   (None, 0.9185265502678995),
   (None, 0.9733334312371444),
   (None, 0.9971321940951793),
   (None, 0.9998334209836808)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [ ]:
import random

def MAP(key: int, value: NamedTuple):
  yield (0, value)

def REDUCE(key: int, rows: Iterator[NamedTuple]):
  yield max(rows)

def RECORDREADER():
    result = []
    for i in range(444):
        result.append((i, random.randint(0, 1024)))
    return result

res = MapReduce(RECORDREADER, MAP, REDUCE)
print(list(res))

[1024]


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [ ]:
import random

def MAP(key: int, value: int):
  yield (0, value)

def REDUCE(key: int, rows: Iterator[NamedTuple]):
  yield sum(rows) / len(rows)

res = MapReduce(RECORDREADER, MAP, REDUCE)
print(list(res))

[494.4684684684685]


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [ ]:
def groupbykey_sort(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  for k2 in t:
    t[k2] = sorted(t[k2])
  return t.items()

print(groupbykey_sort(flatten(map(lambda x: MAP(*x), RECORDREADER()))))

dict_items([(0, [0, 5, 6, 7, 9, 13, 15, 17, 18, 19, 22, 25, 25, 26, 27, 27, 28, 32, 34, 36, 38, 42, 43, 52, 57, 57, 65, 66, 69, 73, 76, 76, 79, 81, 83, 85, 85, 92, 96, 96, 97, 100, 101, 101, 102, 102, 103, 105, 106, 114, 119, 122, 124, 126, 127, 130, 132, 133, 133, 136, 137, 140, 140, 141, 143, 149, 153, 159, 159, 162, 164, 164, 168, 170, 174, 177, 181, 182, 187, 187, 188, 188, 192, 192, 193, 195, 195, 197, 201, 201, 204, 206, 215, 218, 219, 222, 226, 228, 229, 237, 238, 240, 241, 247, 248, 250, 251, 260, 262, 267, 272, 274, 277, 277, 279, 282, 290, 290, 291, 292, 292, 295, 296, 296, 297, 298, 299, 300, 304, 304, 308, 312, 314, 315, 315, 319, 319, 319, 323, 331, 333, 334, 335, 337, 339, 341, 342, 342, 343, 346, 346, 349, 350, 352, 353, 353, 357, 358, 363, 364, 365, 365, 367, 368, 368, 369, 370, 372, 375, 380, 381, 383, 385, 386, 388, 392, 397, 397, 400, 401, 405, 408, 409, 410, 414, 417, 417, 418, 419, 426, 427, 428, 429, 433, 435, 436, 439, 441, 443, 445, 449, 450, 450, 451, 453, 453,

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [ ]:
def MAP(key: int, value: NamedTuple):
  yield (value, value)

def REDUCE(key: int, rows: Iterator[NamedTuple]):
  yield key

res = list(MapReduce(RECORDREADER, MAP, REDUCE))
print(res)
has_duplicates = len(res) != len(set(res))
print('elements is duplicates: ', has_duplicates)

[254, 221, 685, 868, 806, 314, 406, 209, 706, 620, 497, 644, 364, 438, 402, 513, 714, 875, 461, 166, 583, 350, 859, 751, 66, 40, 1022, 125, 207, 3, 340, 661, 788, 515, 223, 569, 761, 236, 587, 726, 96, 613, 677, 813, 69, 471, 54, 424, 658, 222, 926, 80, 941, 168, 135, 218, 383, 289, 622, 13, 916, 675, 235, 757, 834, 282, 793, 392, 122, 395, 669, 562, 713, 56, 26, 9, 164, 780, 44, 371, 891, 388, 415, 171, 81, 290, 285, 429, 37, 259, 445, 91, 241, 710, 610, 233, 893, 543, 1013, 649, 919, 55, 985, 522, 458, 403, 111, 414, 175, 811, 662, 73, 530, 848, 386, 1002, 911, 765, 1005, 173, 196, 619, 373, 912, 255, 573, 719, 466, 422, 92, 741, 151, 657, 197, 696, 499, 469, 854, 888, 948, 419, 397, 441, 576, 439, 723, 432, 23, 489, 572, 574, 299, 452, 325, 621, 650, 920, 366, 842, 296, 840, 478, 586, 492, 100, 534, 410, 165, 79, 770, 695, 421, 42, 198, 592, 551, 774, 245, 959, 929, 511, 387, 268, 904, 760, 681, 1023, 827, 63, 116, 1007, 61, 494, 379, 676, 560, 902, 667, 71, 632, 1000, 640, 484, 795

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [ ]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(key: int, value: NamedTuple):
  if value.gender == 'female':
    yield (value, value)

def REDUCE(key: int, rows: Iterator[NamedTuple]):
  yield rows

def RECORDREADER():
    return [(u.id, u) for u in input_collection]

res = list(MapReduce(RECORDREADER, MAP, REDUCE))
print(res)

[[User(id=1, age=25, social_contacts=240, gender='female')], [User(id=2, age=25, social_contacts=500, gender='female')], [User(id=3, age=33, social_contacts=800, gender='female')]]


### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [ ]:
def MAP(key: int, value: NamedTuple):
  yield (value.gender, value.gender)

def REDUCE(key: int, rows: Iterator[NamedTuple]):
  yield (key, rows)

def RECORDREADER():
    return [(u.id, u) for u in input_collection]

res = list(MapReduce(RECORDREADER, MAP, REDUCE))
print(res)

[('male', ['male']), ('female', ['female', 'female', 'female'])]


### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [ ]:
ls_1 = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800),
    User(id=4, age=16, gender='male', social_contacts=420),
]
ls_2 = [
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=5, age=18, gender='male', social_contacts=322),
    User(id=6, age=23, gender='male', social_contacts=228),
]

def MAP(key: int, value: NamedTuple):
  yield (value, value)

def REDUCE(key: int, rows: Iterator[NamedTuple]):
  yield (key, key)

def RECORDREADER():
    return [(u.id, u) for u in ls_1 + ls_2]

res = list(MapReduce(RECORDREADER, MAP, REDUCE))
print(res)

[(User(id=0, age=55, social_contacts=20, gender='male'), User(id=0, age=55, social_contacts=20, gender='male')), (User(id=1, age=25, social_contacts=240, gender='female'), User(id=1, age=25, social_contacts=240, gender='female')), (User(id=2, age=25, social_contacts=500, gender='female'), User(id=2, age=25, social_contacts=500, gender='female')), (User(id=3, age=33, social_contacts=800, gender='female'), User(id=3, age=33, social_contacts=800, gender='female')), (User(id=4, age=16, social_contacts=420, gender='male'), User(id=4, age=16, social_contacts=420, gender='male')), (User(id=5, age=18, social_contacts=322, gender='male'), User(id=5, age=18, social_contacts=322, gender='male')), (User(id=6, age=23, social_contacts=228, gender='male'), User(id=6, age=23, social_contacts=228, gender='male'))]


### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [ ]:
def MAP(key: int, value: NamedTuple):
  yield (value, value)

def REDUCE(key: int, rows: Iterator[NamedTuple]):
  if len(rows) > 1:
    yield (key, key)

def RECORDREADER():
    return [(u.id, u) for u in ls_1 + ls_2]

res = list(MapReduce(RECORDREADER, MAP, REDUCE))
print(res)

[(User(id=2, age=25, social_contacts=500, gender='female'), User(id=2, age=25, social_contacts=500, gender='female'))]


### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [ ]:
def MAP(key, value):
  if value in ls_1:
    yield (value, 0)
  elif value in ls_2:
    yield (value, 1)

def REDUCE(key, rows):
  if (rows == [0]):
    yield (key, key)

def RECORDREADER():
  return [(u.id, u) for u in ls_1 + ls_2]

res = list(MapReduce(RECORDREADER, MAP, REDUCE))
print(res)

[(User(id=0, age=55, social_contacts=20, gender='male'), User(id=0, age=55, social_contacts=20, gender='male')), (User(id=1, age=25, social_contacts=240, gender='female'), User(id=1, age=25, social_contacts=240, gender='female')), (User(id=3, age=33, social_contacts=800, gender='female'), User(id=3, age=33, social_contacts=800, gender='female')), (User(id=4, age=16, social_contacts=420, gender='male'), User(id=4, age=16, social_contacts=420, gender='male'))]


### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [ ]:
class R(NamedTuple):
  a: int
  b: int

class S(NamedTuple):
  c: str
  b: int

ls_1 = [
    R(a=1, b=1),
    R(a=2, b=2),
    R(a=7, b=2),
    R(a=3, b=5)
]
ls_2 = [
    S(c='sa', b=1),
    S(c='sb', b=2),
    S(c='sc', b=3)
]

def MAP(key, value):
  if value in ls_1:
    yield (value.b, (0, value.a))
  elif value in ls_2:
    yield (value.b, (1, value.c))

def REDUCE(key, rows):
  for r in rows:
    if (r[0] == 0):
      for s in rows:
        if (s[0] == 1):
          yield(key, r[1], s[1])

def RECORDREADER():
  return [(idx, u) for idx, u in enumerate(ls_1 + ls_2)]

res = list(MapReduce(RECORDREADER, MAP, REDUCE))
print(res)

[(1, 1, 'sa'), (2, 2, 'sb'), (2, 7, 'sb')]


### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [ ]:
class Q(NamedTuple):
  a: int
  b: str
  c: int

ls = [
    Q(1, "a", 3),
    Q(2, "a", 2),
    Q(3, "b", 4),
]

def MAP(key, value):
    yield (value.b, value.c)

def REDUCE(key, rows):
    yield (key, max(rows))

def RECORDREADER():
  return [(idx, el) for idx, el in enumerate(ls)]

res = list(MapReduce(RECORDREADER, MAP, REDUCE))
print(res)

[('a', 3), ('b', 4)]


#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [ ]:
import numpy as np
from typing import List

ROWS = 5
COLS = 4
COUNT_OF_PARTS = 2

COLS_IN_PART = COLS // COUNT_OF_PARTS

Matrix = [np.ones((ROWS, COLS_IN_PART)) for _ in range(COUNT_OF_PARTS)]

# Разделил вектор на несколько частей, которые должны поместиться в памяти

vec = [np.random.rand(COLS_IN_PART) for _ in range(COUNT_OF_PARTS)]

def MAP(coordinates:(int, int), value: int):
  i, j = coordinates
  yield (i, value * vec_tmp[j])

def REDUCE(i: int, products: Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  global vec_tmp
  for stripe_index, stripe in enumerate(Matrix):
    vec_tmp = vec[stripe_index]
    for i in range(stripe.shape[0]):
      for j in range(stripe.shape[1]):
        yield ((i, j), stripe[i, j])

print(Matrix)
print(vec)
res = list(MapReduce(RECORDREADER, MAP, REDUCE))
print(res)

[array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]]), array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]])]
[array([0.54817192, 0.02428041]), array([0.49193079, 0.78327993])]
[(0, 1.847663053820144), (1, 1.847663053820144), (2, 1.847663053820144), (3, 1.847663053820144), (4, 1.847663053820144)]


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [ ]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [ ]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(I):
    yield ((i, k), w * small_mat[i, j])


def REDUCE(key, values):
  (i, k) = key
  yield ((i, k), sum(values))

Проверьте своё решение

In [ ]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [ ]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [ ]:
I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

def MAP(k1, v1):
  (matrix_id, row_index, col_index) = k1
  if matrix_id == 0:
    yield (col_index, (matrix_id, row_index, v1))
  else:
    yield (row_index, (matrix_id, col_index, v1))


def REDUCE(key, values):
    first_matrix_values = {}
    second_matrix_values = {}
    for value in values:
        matrix_index, index, val = value
        if matrix_index == 0:
            first_matrix_values[index] = val
        else:
            second_matrix_values[index] = val
    for first_index, first_value in first_matrix_values.items():
        for second_index, second_value in second_matrix_values.items():
            yield ((first_index, second_index), first_value * second_value)

def RECORDREADER():
  for j in range(J):
    for k in range(K):
      yield ((1, j, k), big_mat[j, k])
  for i in range(I):
    for j in range(J):
      yield ((0, i, j), small_mat[i, j])

def MAP_IDENTITY(k1, v1):
  yield (k1, v1)

def REDUCE_AGREGATE(key, values):
  yield (key, sum(values))

def RECORDREADER_IDENTITY(iterable):
  def func():
    for el in iterable:
      yield el
  return func

def assemble_parts(parts_solution):
  parts_solution = list(parts_solution)
  I = max(i for ((i, k), vw) in parts_solution) + 1
  K = max(k for ((i, k), vw) in parts_solution) + 1
  mat = np.empty(shape=(I, K))
  for ((i, k), vw) in parts_solution:
    mat[i, k] = vw
  return mat

np_solution = np.matmul(small_mat, big_mat)
parts_solution = MapReduce(RECORDREADER, MAP, REDUCE)
map_reduce_solution = MapReduce(RECORDREADER_IDENTITY(parts_solution), MAP_IDENTITY, REDUCE_AGREGATE)

np.allclose(np_solution, assemble_parts(map_reduce_solution))

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.




In [ ]:
I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

maps = 2
reducers = 2

def INPUTFORMAT():
  def RECORDREADER(index, matrix):
    rows, cols = matrix.shape
    for i in range(rows):
      for j in range(cols):
        yield ((index, i, j), matrix[i, j])

  yield RECORDREADER(0, small_mat)
  yield RECORDREADER(1, big_mat)

def MAP(k1, v1):
  (matrix_id, row_index, col_index) = k1
  if matrix_id == 0:
    yield (col_index, (matrix_id, row_index, v1))
  else:
    yield (row_index, (matrix_id, col_index, v1))


def REDUCE(key, values):
    first_matrix_values = {}
    second_matrix_values = {}
    for value in values:
        matrix_index, index, val = value
        if matrix_index == 0:
            first_matrix_values[index] = val
        else:
            second_matrix_values[index] = val
    for first_index, first_value in first_matrix_values.items():
        for second_index, second_value in second_matrix_values.items():
            yield ((first_index, second_index), first_value * second_value)

def MAP_IDENTITY(k1, v1):
  yield (k1, v1)

def REDUCE_AGREGATE(key, values):
  yield (key, sum(values))

def GET_INPUTFORMAT(iterable):
  def func():
    for el in iterable:
      yield el[1]
  return func

def assemble_parts(parts_solution):
  parts_solution = list(parts_solution)
  I = max(i for ((i, k), vw) in parts_solution) + 1
  K = max(k for ((i, k), vw) in parts_solution) + 1
  mat = np.empty(shape=(I, K))
  for ((i, k), vw) in parts_solution:
    mat[i, k] = vw
  return mat

np_solution = np.matmul(small_mat, big_mat)

parts_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
parts_output = [(partition_id, list(partition)) for (partition_id, partition) in parts_output]

map_reduce_solution = MapReduceDistributed(GET_INPUTFORMAT(parts_output), MAP_IDENTITY, REDUCE_AGREGATE, COMBINER=None)
map_reduce_solution = [(partition_id, list(partition)) for (partition_id, partition) in map_reduce_solution]

solution_unpacked = []
for value in map_reduce_solution:
  for v in value[1]:
    solution_unpacked.append(v)

np.allclose(np_solution, assemble_parts(solution_unpacked))

126 key-value pairs were sent over a network.
240 key-value pairs were sent over a network.


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [ ]:
I = 2
J = 3
K = 4
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

maps = 2
reducers = 2

def INPUTFORMAT():
    def RECORDREADER(index, matrix, shift):
        rows, cols = matrix.shape
        for i, row in enumerate(matrix):
            for j, val in enumerate(row):
                yield ((index, i + shift, j), val)

    split_size_small = int(np.ceil(small_mat.shape[0] / maps))
    split_size_big = int(np.ceil(big_mat.shape[0] / maps))

    for index, matrix, split_size in [(0, small_mat, split_size_small), (1, big_mat, split_size_big)]:
        for i in range(0, len(matrix), split_size):
            yield RECORDREADER(index, matrix[i:i + split_size], i)

def MAP(k1, v1):
  (matrix_id, row_index, col_index) = k1
  if matrix_id == 0:
    yield (col_index, (matrix_id, row_index, v1))
  else:
    yield (row_index, (matrix_id, col_index, v1))

def REDUCE(key, values):
    first_matrix_values = {}
    second_matrix_values = {}
    for value in values:
        matrix_index, index, val = value
        if matrix_index == 0:
            first_matrix_values[index] = val
        else:
            second_matrix_values[index] = val
    for first_index, first_value in first_matrix_values.items():
        for second_index, second_value in second_matrix_values.items():
            yield ((first_index, second_index), first_value * second_value)

def MAP_IDENTITY(k1, v1):
  yield (k1, v1)

def REDUCE_AGREGATE(key, values):
  yield (key, sum(values))

def GET_INPUTFORMAT(iterable):
  def func():
    for el in iterable:
      yield el[1]
  return func

def assemble_parts(parts_solution):
  parts_solution = list(parts_solution)
  I = max(i for ((i, k), vw) in parts_solution) + 1
  K = max(k for ((i, k), vw) in parts_solution) + 1
  mat = np.empty(shape=(I, K))
  for ((i, k), vw) in parts_solution:
    mat[i, k] = vw
  return mat

np_solution = np.matmul(small_mat, big_mat)

parts_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
parts_output = [(partition_id, list(partition)) for (partition_id, partition) in parts_output]

map_reduce_solution = MapReduceDistributed(GET_INPUTFORMAT(parts_output), MAP_IDENTITY, REDUCE_AGREGATE, COMBINER=None)
map_reduce_solution = [(partition_id, list(partition)) for (partition_id, partition) in map_reduce_solution]

solution_unpacked = []
for value in map_reduce_solution:
  for v in value[1]:
    solution_unpacked.append(v)

np.allclose(np_solution, assemble_parts(solution_unpacked))

18 key-value pairs were sent over a network.
24 key-value pairs were sent over a network.


True